### 1. Importing packages

In [2]:
# pip install openai tiktoken chromadb langchain langchain-community flask GitPython python-dotenv gunicorn

  Using cached flask-3.0.3-py3-none-any.whl.metadata (3.2 kB)
  Using cached GitPython-3.1.43-py3-none-any.whl.metadata (13 kB)
  Using cached anyio-4.4.0-py3-none-any.whl.metadata (4.6 kB)
  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached httpx-0.27.0-py3-none-any.whl.metadata (7.2 kB)
  Using cached pydantic-2.8.2-py3-none-any.whl.metadata (125 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached build-1.2.1-py3-none-any.whl.metadata (4.3 kB)
  Using cached posthog-3.5.0-py2.py3-none-any.whl.metadata (2.0 kB)
  Using cached PyPika-0.48.9.tar.gz (67 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached overrides-7.7.0-py3-none-any.whl.metadata (5.8 kB)
  Using cached importlib_resources-6.4.0-py3-none-any.whl.metadata (3.9 kB)
  Using cached typer-0.12.3-p

In [6]:
import os
from git import Repo
from langchain.text_splitter import Language
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import LanguageParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationSummaryMemory
from langchain.chains import ConversationalRetrievalChain

### 2. Clone Github repositories

#### 2.1. Check current working directory

In [7]:
%pwd

'/Users/prathameshmohite/Desktop/source-code-analysis-1/research'

#### 2.2. Create test repository

In [8]:
!mkdir test_repo

mkdir: test_repo: File exists


#### 2.3. Clone a remote respository using URL

In [10]:
repo_path = "test_repo/"

Repo.clone_from(
    url="https://github.com/entbappy/End-to-end-ML-Project-Implementation", 
    to_path=repo_path
    )

<git.repo.base.Repo '/Users/prathameshmohite/Desktop/source-code-analysis-1/research/test_repo/.git'>

In [11]:
repo_path = "test_repo/"

loader = GenericLoader.from_filesystem(
    repo_path+'/src/mlProject',
    glob = "**/*",
    suffixes=[".py"], # I want to focus on python files only
    parser = LanguageParser(
        language=Language.PYTHON, 
        parser_threshold=500
    )
)

In [12]:
documents = loader.load()

In [13]:
documents

[Document(metadata={'source': 'test_repo/src/mlProject/__init__.py', 'language': <Language.PYTHON: 'python'>}, page_content='import os\nimport sys\nimport logging\n\nlogging_str = "[%(asctime)s: %(levelname)s: %(module)s: %(message)s]"\n\nlog_dir = "logs"\nlog_filepath = os.path.join(log_dir,"running_logs.log")\nos.makedirs(log_dir, exist_ok=True)\n\n\nlogging.basicConfig(\n    level= logging.INFO,\n    format= logging_str,\n\n    handlers=[\n        logging.FileHandler(log_filepath),\n        logging.StreamHandler(sys.stdout)\n    ]\n)\n\nlogger = logging.getLogger("mlProjectLogger")'),
 Document(metadata={'source': 'test_repo/src/mlProject/pipeline/stage_04_model_trainer.py', 'language': <Language.PYTHON: 'python'>}, page_content='from mlProject.config.configuration import ConfigurationManager\nfrom mlProject.components.model_trainer import ModelTrainer\nfrom mlProject import logger\n\n\n\nSTAGE_NAME = "Model Trainer stage"\n\nclass ModelTrainerTrainingPipeline:\n    def __init__(sel

### Chunkings

In [14]:
documents_splitter = RecursiveCharacterTextSplitter.from_language(
    language = Language.PYTHON,
    chunk_size = 2000,
    chunk_overlap = 200 # Thumb rule in langchain community: Chunk overlap should be 10% of total chunk size
)

In [15]:
texts = documents_splitter.split_documents(documents)

In [16]:
len(texts)

19

In [17]:
print(texts[3].page_content)

from mlProject.config.configuration import ConfigurationManager
from mlProject.components.model_evaluation import ModelEvaluation
from mlProject import logger


STAGE_NAME = "Model evaluation stage"

class ModelEvaluationTrainingPipeline:
    def __init__(self):
        pass

    def main(self):
        config = ConfigurationManager()
        model_evaluation_config = config.get_model_evaluation_config()
        model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
        model_evaluation_config.save_results()




if __name__ == '__main__':
    try:
        logger.info(f">>>>>> stage {STAGE_NAME} started <<<<<<")
        obj = ModelEvaluationTrainingPipeline()
        obj.main()
        logger.info(f">>>>>> stage {STAGE_NAME} completed <<<<<<\n\nx==========x")
    except Exception as e:
        logger.exception(e)
        raise e


### Embedding model

In [18]:
# Import the load_dotenv function from the dotenv module to load environment 
# variables from a .env file.
from dotenv import load_dotenv

# Load environment variables from the .env file.
load_dotenv()

# Get the OpenAI API key from the loaded environment variables.
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [19]:
embeddings=OpenAIEmbeddings(disallowed_special=())

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


### Knowledge base (vector DB)

In [20]:
vectordb = Chroma.from_documents(
    texts, 
    embedding=embeddings, 
    persist_directory='./data'
)

vectordb.persist()

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(


### LLM Wrapper

In [21]:
# llm = ChatOpenAI(model_name="gpt-4")
llm = ChatOpenAI()

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [22]:
# For model to remember the context of the entire conversation and save it in memory
memory = ConversationSummaryMemory(
    llm=llm,
    memory_key = "chat_history", 
    return_messages=True
)

In [23]:
qa = ConversationalRetrievalChain.from_llm(
    llm, 
    retriever=vectordb.as_retriever(
        search_type="mmr", # There are many different search types to narrow down your search. 
        # Maximal Marginal Relevance (MMR): MMR is a principle used in information retrieval to prioritize the 
        # selection of documents that are both relevant to a query and diverse from each other. 
        # It aims to balance relevance and diversity in the results returned to the user.
        search_kwargs={"k":3}
    ), 
    memory=memory
)

### Q&A

In [24]:
question = "what is DataIngestion class?"

In [25]:
result = qa(question)
print(result['answer'])

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
Number of requested results 20 is greater than number of elements in index 19, updating n_results = 19


The `DataIngestion` class is a class responsible for downloading and extracting files. It is used in the `DataIngestionTrainingPipeline` class to download and extract files as part of the data ingestion process in a machine learning project. The `DataIngestion` class likely takes a configuration object as a parameter to specify details like source URL, local data file paths, and directories for extracting the downloaded files.


In [26]:
question = "what is ModelTrainer class?"
result = qa(question)
print(result['answer'])

Number of requested results 20 is greater than number of elements in index 19, updating n_results = 19


The `ModelTrainer` class is a part of the mlProject package and is used for training models. It initializes with a configuration object and has a method `train()` that is called to start the training process for the model.


In [27]:
question = "List all the unused libraries"
result = qa(question)
print(result['answer'])

Number of requested results 20 is greater than number of elements in index 19, updating n_results = 19


To identify unused libraries, you would typically analyze the codebase for imports that are not being used anywhere in the code. However, from the code snippets you provided, it seems that all the imported libraries are being used within the defined functions and classes. 

Therefore, based on the code you provided, there are no clearly unused libraries. If there are additional parts of the codebase that you haven't shared where you suspect unused libraries, you would need to review those sections for further analysis.


In [34]:
question = "Should I make any corrections in the entire code repo? Include the exact file name"
result = qa(question)
print(result['answer'])

Number of requested results 20 is greater than number of elements in index 19, updating n_results = 19


There seems to be an inconsistency in the file extension used in the code repo. In the provided code snippets, there is a function `load_json` that is loading JSON files, but the function is actually loading YAML files according to the comments. This discrepancy should be corrected to ensure consistency. 

The file name that might need correction is likely `load_json` because it is loading YAML files based on the comments in the code, while the function name suggests it loads JSON files.
